In [ ]:
#pip install fastdtw
#pip install trjtrypy

In [ ]:
pip install trjtrypy

In [ ]:
import glob
import numpy as np 
import time
import math
import random
from scipy import linalg as LA
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import statsmodels.api as sm
from termcolor import colored

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# install package

In [ ]:
# to make a directory
%cd 'gdrive/My Drive/traj-dist'

/content/gdrive/My Drive/traj-dist


In [ ]:
# to see in what directory we are in

In [ ]:
pwd

'/content/gdrive/My Drive/traj-dist'

In [ ]:
# to install setup.py from the current directory
!python setup.py install

/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/lib/python3.7/distutils/dist.py:274: UserWarning: Unknown distribution option: 'download_urt'
  warnings.warn(msg)
running install
running bdist_egg
running egg_info
writing traj_dist.egg-info/PKG-INFO
writing dependency_links to traj_dist.egg-info/dependency_links.txt
writing requirements to traj_dist.egg-info/requires.txt
writing top-level names to traj_dist.egg-info/top_level.txt
adding license file 'LICENSE.txt'
writing manifest file 'traj_dist.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
running build_ext
skipping 'traj_dist/cydist/basic_geographical.c' Cython extension (up-to-date)
skipping 'traj_dist/cydist/basic_euclidean.c' Cython extension (up-to-date)
skipping

In [ ]:
!pip install geohash2

# "######################" Now restart runtime "######################"

But don't run the above block

In [ ]:
import traj_dist.distance as tdist
import pickle

# Preprocessing data

In [ ]:
def read_file(file_name):
    data = []
    with open(file_name, "r") as f:
        for line in f:
            item = line.strip().split(",")
            data.append(np.array(item))
    return np.array(data)

data1 = read_file('/content/gdrive/My Drive/Colab Notebooks/go_track_tracks.csv')[1:,:-1]
data2 = read_file('/content/gdrive/My Drive/Colab Notebooks/go_track_trackspoints.csv')[1:,1:4]

float1 = np.vectorize(float)
int1 = np.vectorize(int)
track_id = int1(data1[:,0])
labels = np.where(int1(data1[:,-1]) < 1.5, int1(data1[:,-1]), -1) 
traj = float1(data2)

trajec = [0] * 163

for i in range(163):
    trajec[i] = []
    I = np.where(traj[:,2] == track_id[i])
    trajec[i] = np.array([labels[i], traj[I]], dtype = 'object')

trajec = np.array(trajec)

trajectory = [0] * 163
trajectory_label_id = [0] * 163

for i in range(163):
    trajectory[i] = trajec[i][1][:,:2]
    trajectory_label_id[i] = np.array([trajec[i][1][:,:2], trajec[i][0], trajec[i][1][:,2][0]], dtype = 'object')
    
trajectory_label_id = np.array(trajectory_label_id, dtype = 'object')
trajectory = np.array(trajectory, dtype = 'object')


min_length = 10
max_length = 1000 #160 for balance data
l = 0
index = [] 
for i in range(163):
    if len(trajectory[i]) < min_length or len(trajectory[i]) > max_length:
        l = l + 1
    else:
        index.append(i)
        
print("l, 163-l=", l, 163-l)

trajectories = [0] * (163-l)
trajectories_label_id = [0] * (163-l)

j = 0
for i in range(163):
    if len(trajectory[i]) >= min_length and len(trajectory[i]) <= max_length:
        trajectories[j] = np.array(trajectory[i])
        trajectories_label_id[j] = trajectory_label_id[i]
        j = j + 1

trajectories_label_id = np.array(trajectories_label_id, dtype = 'object')
trajectories = np.array(trajectories, dtype = 'object')

cars = trajectories_label_id[np.where(trajectories_label_id[:,1] == 1)][:,:2][:,0]
buses = trajectories_label_id[np.where(trajectories_label_id[:,1] == -1)][:,:2][:,0]

l, 163-l= 40 123


In [ ]:
def remove_segments(traj): # removes stationary points
    p2 = traj[1:]
    p1 = traj[:-1]
    L = ((p2-p1)*(p2-p1)).sum(axis =1)
    I = np.where(L>1e-16)[0]
    return traj[I]

In [ ]:
cars = np.array(list(map(remove_segments, cars)), dtype='object')
buses = np.array(list(map(remove_segments, buses)), dtype='object')

cars_copy = cars.copy()
buses_copy = buses.copy()
print("len(cars), len(buses)=", len(cars), len(buses))

len(cars), len(buses)= 78 45


In [ ]:
cars_11 = cars[11].copy()
cars_12 = cars[12].copy()

In [ ]:
a = np.arange(len(cars))
I = np.where((a != 11) & (a != 12) & (a != 28) & (a != 29))
cars = cars[I]
buses = buses[:-1]
len(cars), len(buses)

(74, 44)

In [ ]:
cars = np.array([cars_11, cars_12] + list(cars), dtype='object')
cars.shape

(76,)

In [ ]:
a, c = np.min((np.min([np.min(cars[i], axis=0) for i in range(len(cars))], axis=0), 
       np.min([np.min(buses[i], axis=0) for i in range(len(buses))], axis=0)), axis=0)
  
b, d = np.max((np.max([np.max(cars[i], axis=0) for i in range(len(cars))], axis=0), 
               np.max([np.max(buses[i], axis=0) for i in range(len(buses))], axis=0)), axis=0)

m = 20
Q = np.ones((m,2))

Q[:,0] = (b - a + 0.001) * np.random.random_sample(m) + a - 0.02
Q[:,1] = (d - c + 0.001) * np.random.random_sample(m) + c - 0.01

In [ ]:
def get_mu(data_1, data_2):
    a = np.mean([np.mean(data_1[i], 0) for i in range(len(data_1))], 0)
    b = np.mean([np.mean(data_2[i], 0) for i in range(len(data_2))], 0)
    c = abs(a-b)
    return max(c)

In [ ]:
def error(A,B):
    return abs(A - B) / (abs(A) + abs(B))

# From github page: https://github.com/bguillouet/traj-dist

It includes 9 distances for trajectories including: Continuous Frechet, Discrete Frechet, Hausdorff, DTW, SSPD, LCSS, EDR, ERP.

## All but the continuous Frechet distance are really fast.

#### Calculate distances all together

In [ ]:
def calculate_dists(data1, data2, metrics, eps_lcss=0.02, eps_edr=0.02): 
    ''' eps_lcss: if metric = 'lcss', then eps_lcss is effective.
        eps_edr: if metric = 'edr', then eps_edr is effective.
        id: 1 if car-bus dataset is used and 2 if the generated data_1 and data_2 are used'''
    start_time = time.time() 
    data = np.concatenate((data1, data2), 0)
    n = len(data)
    for metric in metrics:
        if metric == 'lcss':
            A = tdist.pdist(data, metric, type_d="euclidean", eps=eps_lcss)
        elif metric == 'edr':
            A = tdist.pdist(data, metric, type_d="euclidean", eps=eps_edr)
        else: 
            A = tdist.pdist(data, metric)
        tri = np.zeros((n, n))
        tri[np.triu_indices(n, 1)] = A
        for i in range(1, n):
            for j in range(i):
                tri[i][j] = tri[j][i]
        np.savetxt('/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (car-bus)/'+metric+'-76-44.csv', 
                    tri, delimiter=',')

    total_time = time.time() - start_time
    return total_time

In [ ]:
Metrics = ['discret_frechet', 'hausdorff', 'dtw', 'sspd', 'lcss', 'edr', 'erp']

calculate_dists(cars, buses, metrics=Metrics, eps_lcss=0.02, eps_edr=0.02)

/usr/local/lib/python3.7/dist-packages/traj_dist-1.1-py3.7-linux-x86_64.egg/traj_dist/distance.py:552: UserWarning: g parameter should be specified for metric erp. Default is 
  warnings.warn("g parameter should be specified for metric erp. Default is ")


125.63468956947327

In [ ]:
#Metrics = ['frechet']

#calculate_dists(cars, buses, metrics=Metrics)

33787.003963947296

#### Calculate distances for continuous Frechet line by line by the following function. Then at the end concatenate them together.

In [ ]:
# data = np.concatenate((data1, data2), 0)
def calculate_dists_line_by_line_frechet(data, path, start_idx, end_idx): 
    n = len(data)
    for i in range(start_idx, end_idx):
        start_time = time.time()
        def f(traj):
            return tdist.frechet(data[i], traj)
        A = list(map(f, data[i+1:]))
        np.savetxt(path+str(i)+'.csv', A, delimiter=',') 
        total_time = time.time() - start_time
        print(f"time for step {i}: {total_time}")
    return total_time

In [ ]:
path = '/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (car-bus)/frechet lines/'
data = np.concatenate((cars, buses), 0) 
calculate_dists_line_by_line_frechet(data, path, start_idx=0, end_idx=len(data)-1)

### To stack the 1d generated vectors together to get the distance matrix of continuous Frechet distance we will use the following get_matrix function.

In [ ]:
def get_matrix(data1, data2, path):
    '''
    path is: 
    '/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (car-bus)/frechet lines/'
    or
    '/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (car-bus-generated)/frechet lines/'
    '''
    n = len(data1) + len(data2)
    matrix = np.zeros((n,n))
    for i in range(n-1):
        #matrix[i][i+1:] = np.array(pd.read_csv(path+str(i)+'.csv', header=None)).reshape(n-i-1,)
        matrix[i][i+1:] = np.loadtxt(path+str(i)+'.csv')
    for i in range(1, n):
        for j in range(i):
            matrix[i][j] = matrix[j][i]
    np.savetxt(path[:-14]+'frechet.csv', matrix, delimiter=',')
    return matrix

In [ ]:
path = '/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (car-bus)/frechet lines/'
A = get_matrix(cars, buses, path)
A.shape

(118, 118)

# KNN with 8 distances: 

### Continuous Frechet, Discrete Frechet, Hausdorff, DTW, LCSS, SSPD, EDR, ERP

In [ ]:
def KNN_with_dists(n_1, n_2, dists_names, paths_to_dists):
    '''path example: '/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (car-bus)/sspd.csv'
       dists_names: a list of distance names
       paths_to_dists: the paths list to the corresponding distancas (each path 
                       points out to the corresponding distance matrix)
       n_1: len(data_1)
       n_2: len(data_2)
       dist_name: the name of distance used to calculate sitance matrix 
       (the name is taken from a list above called metrics)'''

    train_errors = np.zeros(len(dists_names))
    test_errors = np.zeros(len(dists_names))

    I_1, J_1, y_train_1, y_test_1 = train_test_split(np.arange(n_1), 
                                                np.ones(n_1), test_size=0.3)
    I_2, J_2, y_train_2, y_test_2 = train_test_split(np.arange(n_1, n_1+n_2), 
                                                np.ones(n_2), test_size=0.3)
    labels = np.array([1]*n_1 + [0] * n_2)
    I = np.concatenate((I_1, I_2), 0)
    np.random.shuffle(I)
    J = np.concatenate((J_1, J_2), 0)
    np.random.shuffle(J)

    for i in range(len(dists_names)):

        dist_matrix = np.array(pd.read_csv(paths_to_dists[i],  header=None))

        D_train = dist_matrix[I][:, I]
        D_test = dist_matrix[J][:,I]
        train_labels = labels[I]
        test_labels = labels[J]

        clf = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
        
        #Train the model using the training sets
        clf.fit(D_train, list(train_labels))

        #Predict labels for train dataset
        train_pred = clf.predict(D_train)
        train_errors[i] = sum(train_labels != train_pred)/len(I)
        
        #Predict labels for test dataset
        test_pred = clf.predict(D_test)
        test_errors[i] = sum((test_labels != test_pred))/len(J)
        
    return train_errors, test_errors

In [ ]:
def KNN_average_error(data1, data2, num_trials, dists_names, paths_to_dists):

    '''dists_names: a list of distance names
       paths_to_dists: the paths list to the corresponding distancas (each path 
                       points out to the corresponding distance matrix)'''

    Start_time = time.time()

    train_errors = np.zeros((num_trials, len(dists_names)))
    test_errors = np.zeros((num_trials, len(dists_names)))

    for i in range(num_trials):
        tr_errors, ts_errors = KNN_with_dists(len(data1), len(data2), dists_names, paths_to_dists)
        train_errors[i] = tr_errors
        test_errors[i] = ts_errors

    train_error = np.mean(train_errors, axis=0)
    test_error = np.mean(test_errors, axis=0)
    std_test_error = np.std(test_errors, axis=0)

    Dict = {}
    for i in range(len(dists_names)):
        Dict[i+1] = [f"KNN with {dists_names[i]}", 
                     np.round(train_error[i], decimals = 4), 
                     np.round(test_error[i], decimals = 4), 
                     np.round(std_test_error[i], decimals = 4)]

    df = pd.DataFrame.from_dict(Dict, orient='index', columns=['Classifier',
                                'Train Error', 'Test Error', 'std'])
    print(colored(f"num_trials = {num_trials}", "blue"))
    print(colored(f'total time = {time.time() - Start_time}', 'green'))

    return (df, train_errors, test_errors, train_error, test_error)

In [ ]:
Metrics = ['frechet', 'discret_frechet', 'hausdorff', 'dtw', 'lcss', 'sspd', 'edr', 'erp']

In [ ]:
paths = []
eps = 0.02
for i in range(len(Metrics)):
    paths.append('/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (car-bus)/'+Metrics[i]+'-76-44.csv')

F = KNN_average_error(cars, buses, num_trials=50, dists_names=Metrics, paths_to_dists=paths)
F[0]

num_trials = 50
total time = 8.637168407440186


,Classifier,Train Error,Test Error,std
1,KNN with frechet,0.1733,0.2573,0.0633
2,KNN with discret_frechet,0.1720,0.2557,0.0642
3,KNN with hausdorff,0.1513,0.2405,0.0663
4,KNN with dtw,0.1583,0.2368,0.0566
5,KNN with lcss,0.2588,0.3022,0.0677
6,KNN with sspd,0.1398,0.2200,0.0465
7,KNN with edr,0.2140,0.3124,0.0774
8,KNN with erp,0.2224,0.3335,0.0555


# KNN with fastdtw which is an approximate dtw

In [ ]:
pip install fastdtw

In [ ]:
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

In [ ]:
dataa = np.concatenate((cars, buses), 0)
fastdtw_matrix = np.array([np.array([fastdtw(dataa[i], dataa[j])[0] for j in range(len(dataa))]) for i in range(len(dataa))])
fastdtw_matrix.shape

(120, 120)

In [ ]:
def KNN_with_fastdtw(n_1, n_2, dist_matrix):
    '''n_1: len(data_1)
       n_2: len(data_2)
       dist_name: the name of distance used to calculate sitance matrix 
       (the name is taken from a list above called metrics)'''

    I_1, J_1, y_train_1, y_test_1 = train_test_split(np.arange(n_1), 
                                                np.ones(n_1), test_size=0.3)
    I_2, J_2, y_train_2, y_test_2 = train_test_split(np.arange(n_1, n_1+n_2), 
                                                np.ones(n_2), test_size=0.3)
    labels = np.array([1]*n_1 + [0] * n_2)
    I = np.concatenate((I_1, I_2), 0)
    np.random.shuffle(I)
    J = np.concatenate((J_1, J_2), 0)
    np.random.shuffle(J)

    D_train = dist_matrix[I][:, I]
    D_test = dist_matrix[J][:,I]
    train_labels = labels[I]
    test_labels = labels[J]

    clf = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
    
    #Train the model using the training sets
    clf.fit(D_train, list(train_labels))

    #Predict labels for train dataset
    train_pred = clf.predict(D_train)
    train_errors = sum(train_labels != train_pred)/len(I)
    
    #Predict labels for test dataset
    test_pred = clf.predict(D_test)
    test_errors = sum((test_labels != test_pred))/len(J)
        
    return train_errors, test_errors

In [ ]:
def KNN_fastdtw_average_error(data1, data2, dist_matrix, num_trials):

    '''dists_names: a list of distance names
       paths_to_dists: the paths list to the corresponding distancas (each path 
                       points out to the corresponding distance matrix)'''

    Start_time = time.time()

    train_errors = np.zeros(num_trials)
    test_errors = np.zeros(num_trials)

    for i in range(num_trials):
        tr_errors, ts_errors = KNN_with_fastdtw(len(data1), len(data2), dist_matrix)
        train_errors[i] = tr_errors
        test_errors[i] = ts_errors

    train_error = np.mean(train_errors)
    test_error = np.mean(test_errors)
    std_test_error = np.std(test_errors)

    Dict = {}
    Dict[1] = [f"KNN with fastdtw", np.round(train_error, decimals = 4), 
               np.round(test_error, decimals = 4), 
               np.round(std_test_error, decimals = 4)]

    df = pd.DataFrame.from_dict(Dict, orient='index', columns=['Classifier',
                                'Train Error', 'Test Error', 'std'])
    print(colored(f"num_trials = {num_trials}", "blue"))
    print(colored(f'total time = {time.time() - Start_time}', 'green'))

    return (df, train_errors, test_errors, train_error, test_error)

In [ ]:
E = KNN_fastdtw_average_error(cars, buses, dist_matrix=fastdtw_matrix, num_trials=50)
E[0]

num_trials = 50
total time = 0.2895853519439697


,Classifier,Train Error,Test Error,std
1,KNN with fastdtw,0.1571,0.2438,0.0702


# KNN with LSH

In [ ]:
pip install trjtrypy

In [ ]:
from google.colab import files
files.upload()

#import KNN_with_LSH_class
from KNN_with_LSH_class import KNN_with_LSH

Saving KNN_with_LSH_class.py to KNN_with_LSH_class.py


In [ ]:
KNN_with_LSH_class = KNN_with_LSH(cars, buses, number_circles=20, num_trials=50)
KNN_with_LSH_class.KNN_LSH_average_error()[0]

num_trials = 50
total time = 4.229677677154541


,Classifier,Train Error,Test Error,std
1,KNN with LSH,0.2434,0.2968,0.0931


# KNN with soft-dtw is implemented in Anaconda as the package couldn't be installed on Google Colab.